# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1631937024592_0314'

In [3]:
pd.options.display.max_columns = 35

In [4]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831', '8FBD43EF0885489AA9FF961D66294839'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 
defaultcontext2 = '8FBD43EF0885489AA9FF961D66294839'

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831
8FBD43EF0885489AA9FF961D66294839


In [5]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [6]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [7]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='Coronavirus'
print(algorithmName)

365
365
365
user_rc047407
Coronavirus


In [8]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [10]:
spark.sql("""select * from lab """).printSchema()

root
 |-- labid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- labcode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- loincclass: string (nullable = true)
 |-- type: string (nullable = true)
 |-- servicedate: string (nullable = true)
 |-- serviceperiod: struct (nullable = true)
 |    |-- startDate: string (nullable = true)
 |    |-- endDate: string (nullable = true)
 |-- typedvalue: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- textValue: 

In [11]:
spark.sql("""select * from questionnaire """).printSchema()

root
 |-- questionnaireid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- status: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |  

In [12]:
spark.sql("""select distinct tenant
                    ,personId

                from demographics
                
                order by 2,1
             
                limit(25)
                """).toPandas()

,tenant,personId
0,13,00000007-4c76-45de-8b82-8052b3564944
1,40,00000077-3603-4025-bb94-2cb28f0232e2
2,77,0000008b-4c47-498d-b0db-4dcbad1e4e84
3,37,000000a7-0c12-4af2-abd9-dedeb2c59d7b
4,6,000000c5-0fec-4cb7-b397-fb88e4a0851b
5,45,00000116-2fb9-4b5a-8ce2-4a3ce130ba90
6,56,00000137-d832-47c9-a3a0-4e7c2e6ecd75
7,77,00000142-71f8-4345-8cbc-5ed346a2bfa1
8,35,00000240-2272-4c71-b7e5-11f6e04c68b4
9,44,00000258-0a18-49d2-b7f8-c13790a2ae99


In [13]:
spark.sql("""select gender.standard.primaryDisplay as gender
                    ,birthsex.standard.primaryDisplay as birthsex
                    ,maritalstatus.standard.primaryDisplay as maritalstatus
                    ,races.standard.primaryDisplay as races
                    ,zip_code
                    ,birthDate
                from demographics
             
                limit(25)
                """).toPandas()

,gender,birthsex,maritalstatus,races,zip_code,birthDate
0,Female,None,Never Married,[White],0,1991-11-05T00:00:00
1,Male,None,Never Married,[Other Race],0,1997-02-16T00:00:00
2,Male,None,Never Married,[White],0,1997-12-09T00:00:00
3,Male,None,Never Married,None,0,1986-09-18T00:00:00
4,Female,None,Never Married,[White],0,1993-10-05T00:00:00
5,Female,None,None,None,3,1984-08-30T00:00:00
6,Male,None,Never Married,[Unknown racial group],0,2009-05-27T00:00:00
7,Female,None,None,None,0,1980-12-03T00:00:00
8,Male,None,None,None,,1966-12-03T00:00:00
9,Female,None,Never Married,[White],0,2016-02-14T00:00:00


In [14]:
spark.sql("""select * from demographics""").printSchema()

root
 |-- personid: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- dateofdeath: string (nullable = true)
 |-- gender: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- birthsex: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |

In [15]:
spark.sql("""select * from encounter""").printSchema()

root
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- reasonforvisit: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- financialclass: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullabl

In [16]:
spark.sql("""select * from procedure""").printSchema()

root
 |-- procedureid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- procedurecode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- modifiercodes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- standard: struct (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |    |    |-- standardCodings: arr

In [17]:
spark.sql("""select * from medication""").printSchema()

root
 |-- medicationid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- intendeddispenser: string (nullable = true)
 |-- startdate: string (nullable = true)
 |-- intendedadministrator: string (nullable = true)
 |-- doseunit: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- stopdate: string (nullable = true)
 |-- category: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (n

In [18]:
spark.sql("""select * from tenant_attributes """).printSchema()

root
 |-- tenant: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- bed_size: string (nullable = true)
 |-- speciality: string (nullable = true)
 |-- segment: string (nullable = true)



In [19]:
spark.sql("""select * from tenant_attributes order by 1""").toPandas()

,tenant,zip_code,bed_size,speciality,segment
0,1,5,500-999,IDN/Regional Health Authority,Regional Hospital
1,10,6,500-999,Hospital,Regional Hospital
2,100,4,<100,Critical Access Hospital,Community Healthcare
3,101,2,200-299,Hospital,Childrens
4,102,6,<100,Community Hospital,Critical Access
...,...,...,...,...,...
96,95,7,<100,Critical Access Hospital,Community Healthcare
97,96,6,500-999,Academic Medical Center,Academic
98,97,6,<100,Critical Access Hospital,Community Healthcare
99,98,6,<100,Hospital,Community Healthcare


####  ZIP Codes Beginning With	States
    0	Connecticut, Massachusetts, Maine, New Hampshire, New Jersey, Puerto Rico, Rhode Island, Vermont, Virgin Islands, Army Post Office Europe, Fleet Post Office Europe
    1	Delaware, New York, Pennsylvania
    2	District of Columbia, Maryland, North Carolina, South Carolina, Virginia, West Virginia
    3	Alabama, Florida, Georgia, Mississippi, Tennessee, Army Post Office Americas, Fleet Post Office Americas
    4	Indiana, Kentucky, Michigan, Ohio
    5	Iowa, Minnesota, Montana, North Dakota, South Dakota, Wisconsin
    6	Illinois, Kansas, Missouri, Nebraska
    7	Arkansas, Louisiana, Oklahoma, Texas
    8	Arizona, Colorado, Idaho, New Mexico, Nevada, Utah, Wyoming
    9	Alaska, American Samoa, California, Guam, Hawaii, Marshall Islands, Federated States of Micronesia, Northern Mariana Islands, Oregon, Palau, Washington, Army Post Office Pacific, Fleet Post Office Pacific

In [22]:
covidPatientDf = spark.sql(f"""


    SELECT distinct
        e.tenant
        ,t.bed_size as tenant_bed_size
        ,t.speciality as tenant_speciality
        ,t.segment as tenant_segment
        ,t.zip_code as tenant_zip_code
        ,e.personId
        ,e.encounterId
        ,e.serviceDate
        ,e.dischargeDate
        ,d.zip_code as zip_code
        ,if (d.deceased = true, 1, 0) as deceased
       ,ROUND((datediff(date_add(current_date(), - 0), d.birthDate))/365,0) as age
       ,CASE  
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(d.gender, array('FEMALE_GEN'),'{defaultcontext}' ) THEN 'female' 
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(d.gender, array('MALE_GEN'),'{defaultcontext}' ) THEN 'male'
            ELSE 'unknown'
         END as gender 
        ,CASE  
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('INPATIENT_VISIT_ENC'),'{defaultcontext}' ) THEN 'inpatient visit' 
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('OUTPATIENT_VISIT_ENC'),'{defaultcontext}' ) THEN 'outpatient visit'
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('EMERGENCY_VISIT_ENC'),'{defaultcontext}' ) THEN 'emergency visit'
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('OBSERVATION_VISIT_ENC'),'{defaultcontext}' ) THEN 'observation visit'
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('LONG_TERM_CARE_FACILITY_ENC'),'{defaultcontext}' ) THEN 'long term care visit'
            WHEN HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('HOME_VISIT_ENC'),'{defaultcontext}' ) THEN 'home visit'
            ELSE 'other visit'
         END as encounter_type 
         

        FROM {db}.encounter e

        JOIN {db}.demographics d
            ON e.personId = d.personId
              and e.tenant = d.tenant
               
        JOIN {db}.tenant_attributes t
            ON  e.tenant = t.tenant            
                                   
        WHERE  e.dischargeDate is not null
            and e.serviceDate <= e.dischargeDate
            
            and  ( exists  (   SELECT 1 
                               FROM {db}.condition c
                                WHERE e.personId = c.personId
                                  and e.encounterId = c.encounterId
                                  and e.tenant = c.tenant
                                  and ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                                        'CORONAVIRUS_COVID_19_POSITIVE_CLIN', 'CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN',
                                        'CORONAVIRUS_COVID_19_TRAVEL_TO_AREA_AFFECTED_CLIN'), '{defaultcontext}' ) )
                            )
                or exists (   SELECT 1 
                              FROM {db}.lab r
                               WHERE e.encounterId = r.encounterId
                                  and HAS_ANY_CONCEPT_IN_CONTEXT(r.labCode, array('SARS_CORONAVIRUS_2_COVID_19_OBSTYPE'), '{defaultcontext}' )    ---  SARS_CORONAVIRUS_2_COVID_19_OBSTYPE  Negative
                                            AND ( has_any_concept_in_context(r.typedvalue.codifiedValues.values.value,
                                               array('POSITIVE_QUAL'), 
                                                    '5E259FD575B54D4982D32D4E92DCA831' )
                                              )    
                
                            )            
                 )      
       

""")

In [23]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_CovidPatient""").show() 

++
||
++
++



In [24]:
covidPatientDf.write.saveAsTable(f"""{userName}.{algorithmName}_CovidPatient """)

In [25]:
spark.sql(f"""select *
   from {userName}.{algorithmName}_CovidPatient
   limit (50)
    """).toPandas()

,tenant,tenant_bed_size,tenant_speciality,tenant_segment,tenant_zip_code,personId,encounterId,serviceDate,dischargeDate,zip_code,deceased,age,gender,encounter_type
0,66,500-999,Hospital,IDN,2,0005f305-132a-44d7-8f7e-eb4ef818d37c,a98d408f-23df-4304-8d18-e3dda4462ac6,2021-01-21T20:07:12+00:00,2021-01-26T04:59:59+00:00,,0,26.0,male,other visit
1,66,500-999,Hospital,IDN,2,0005f305-132a-44d7-8f7e-eb4ef818d37c,a98d408f-23df-4304-8d18-e3dda4462ac6,2021-01-21T20:07:12+00:00,2021-01-26T04:59:59+00:00,2,0,26.0,male,other visit
2,54,300-499,IDN/Regional Health Authority,IDN,2,0006c562-34d1-4d96-a8b6-a9d5b3c592cb,0a2da532-f590-4ea1-bfbf-e15e6e97308b,2021-01-23T00:00:00,2021-01-23T00:00:00,,0,24.0,female,other visit
3,54,300-499,IDN/Regional Health Authority,IDN,2,0006c562-34d1-4d96-a8b6-a9d5b3c592cb,0a2da532-f590-4ea1-bfbf-e15e6e97308b,2021-01-23T00:00:00,2021-01-23T00:00:00,,0,NaN,unknown,other visit
4,54,300-499,IDN/Regional Health Authority,IDN,2,0006c562-34d1-4d96-a8b6-a9d5b3c592cb,0a2da532-f590-4ea1-bfbf-e15e6e97308b,2021-01-23T00:00:00,2021-01-23T00:00:00,2,0,24.0,female,other visit
5,61,500-999,IDN/Regional Health Authority,IDN,0,000935a2-60f2-46e9-a5a2-0aa8a28966c2,df8295d6-03dd-4955-bf4c-a2547e1164ba,2020-05-16T13:06:00+00:00,2020-05-17T03:59:59+00:00,0,0,38.0,female,outpatient visit
6,78,300-499,Hospital,Regional Hospital,7,000cec85-a999-47b6-b62a-a1fc505412f5,48bc5af9-ac8b-4107-831f-cf8263c92bb5,2021-01-10T21:52:40+00:00,2021-01-11T20:19:00+00:00,7,0,37.0,male,emergency visit
7,45,>=1000,IDN/Regional Health Authority,IDN,1,000de908-a0dc-4c6e-9386-070f6893ccbb,7225163e-1dce-4ede-8e75-ab6023c9f36b,2020-12-06T07:41:00+00:00,2020-12-07T23:25:00+00:00,9,0,27.0,female,inpatient visit
8,63,>=1000,IDN/Regional Health Authority,IDN,8,0010bac9-8b0f-455f-9088-3399712245e0,024152f4-c7d8-4565-a1f6-78fff432d137,2020-08-25T00:56:36+00:00,2020-08-26T06:59:59+00:00,8,0,31.0,female,other visit
9,75,<100,Hospital,Regional Hospital,5,001241f0-ea16-431c-862e-02ada47e94f5,b0dd99d6-6383-43ff-b9ae-8823de575e80,2020-11-19T19:34:12+00:00,2020-11-19T21:36:35+00:00,5,0,50.0,male,emergency visit


In [26]:
spark.sql(f"""select count(*)
   from {userName}.{algorithmName}_CovidPatient

    """).toPandas()

,count(1)
0,5284473
